In [1]:
import os
import pickle
import pandas as pd
from llama_cpp import Llama
import json
import time

DATA_PATH = "/Users/zanchitta/Developer/BugDetectiveAI/Bugdetectiveai/data/pytracebugs_dataset_v1"

# print("Data path:", DATA_PATH)
BUGGY_DATAPATH = DATA_PATH + "/buggy_dataset"
STABLE_DATAPATH = DATA_PATH + "/stable_dataset"
bugfixes_test_path = BUGGY_DATAPATH + "/bugfixes_test.pickle"
bugfixes_train_path = BUGGY_DATAPATH + "/bugfixes_train.pickle"
bugfixes_valid_path = BUGGY_DATAPATH + "/bugfixes_valid.pickle"

stable_code_test_path = STABLE_DATAPATH + "/stable_code_test.pickle"
stable_code_train_path = STABLE_DATAPATH + "/stable_code_train.pickle"
stable_code_valid_path = STABLE_DATAPATH + "/stable_code_valid.pickle"

In [2]:
# listing existing models for us to work with
!ls ../../data/models

Ministral-3b-instruct.Q2_K.gguf


In [3]:
# read files and convert to dataframes
bugfixes_test = pd.read_pickle(bugfixes_test_path)
bugfixes_train = pd.read_pickle(bugfixes_train_path)
bugfixes_valid = pd.read_pickle(bugfixes_valid_path)

stable_code_test = pd.read_pickle(stable_code_test_path)
stable_code_train = pd.read_pickle(stable_code_train_path)
stable_code_valid = pd.read_pickle(stable_code_valid_path)
bugfixes_train["input_length"] = bugfixes_train["before_merge"].apply(lambda x: len(x))
bugfixes_train = bugfixes_train[bugfixes_train["input_length"] < 900]

In [44]:
llm = Llama(
    model_path="/Users/zanchitta/Developer/BugDetectiveAI/Bugdetectiveai/data/models/Ministral-3b-instruct.Q2_K.gguf",
    n_ctx=2048,
    n_threads=8,  # check in the future for more details into this
    verbose=False,
)

llama_new_context_with_model: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_f32_f32                    (not supported)
ggml_metal_init: skipping kernel_mul_mm_f16_f32                    (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_q4_0_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_q

In [7]:
sample_data = bugfixes_train.sample(1, random_state=2)
sample_data.reset_index(drop=False, inplace=True)
easy_test = """def last_element_of_array(array):
    return array[len(array)]
"""
sample_data

,index,after_merge,before_merge,filename,full_file_code_after_merge,full_file_code_before_merge,function_name,url,source code and errors,full_traceback,traceback_type,before_merge_without_docstrings,after_merge_without_docstrings,before_merge_docstrings,after_merge_docstrings,path_to_snippet_before_merge,path_to_snippet_after_merge,input_length
0,568352,"def StartUdpServer(context=None, identity=None...","def StartUdpServer(context=None, identity=None...",pymodbus/server/sync.py,"""""""\nImplementation of a Threaded Modbus Serve...","""""""\nImplementation of a Threaded Modbus Serve...",StartUdpServer,https://github.com/riptideio/pymodbus/issues/377,"[{'piece_type': 'error message', 'piece_conten...","client = ModbusSerialClient(port='notexist', m...",AttributeError,"def StartUdpServer(context=None, identity=None...","def StartUdpServer(context=None, identity=None...",[A factory to start and run a udp modbus serve...,[A factory to start and run a udp modbus serve...,buggy_snippets_files/bdcd76ade2414e1804de3c6bc...,buggy_snippets_files/bdcd76ade2414e1804de3c6bc...,661


In [48]:
# print("Before Merge")
# print(sample_data.loc[0,'before_merge'])
# print("############################################")
# print("After Merge")
# print(sample_data.loc[0,'after_merge'])
# print("############################################")
# print("Source Code and Errors")
# print(sample_data.loc[0,'source code and errors'])


In [8]:
print(sample_data.loc[0, "after_merge"])

def StartUdpServer(context=None, identity=None, address=None,
                   custom_functions=[], **kwargs):
    """ A factory to start and run a udp modbus server

    :param context: The ModbusServerContext datastore
    :param identity: An optional identify structure
    :param address: An optional (interface, port) to bind to.
    :param custom_functions: An optional list of custom function classes
        supported by server instance.
    :param framer: The framer to operate with (default ModbusSocketFramer)
    :param ignore_missing_slaves: True to not send errors on a request
                                    to a missing slave
    """
    framer = kwargs.pop('framer', ModbusSocketFramer)
    server = ModbusUdpServer(context, framer, identity, address, **kwargs)
    for f in custom_functions:
        server.decoder.register(f)
    server.serve_forever()


In [105]:
easy_test = """def last_element_of_array(array):
    return array[len(array)]
"""

## Define System message and User prompt

In [45]:
system_message = " You are a helping tearcher assistant. Your job is to explain codes, and not rewrite code."
prompt = f"Tell me if the following code contains a bug or not. In case of bug, explain the bug.\n\n<code>\n{sample_data.loc[0, 'before_merge']}\n<\code>"

## Execute model inference on Sample Data

In [102]:
def generate_response(before_merge_text):
    system_message = " You are a helping tearcher assistant. Your job is to explain codes, and not rewrite code."
    prompt = f"Tell me if the following code contains a bug or not. In case of bug, explain the bug.\n\n<code>\n{before_merge_text}\n<\code>"
    try:
        resp = llm.create_chat_completion(
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": prompt},  # Use before_merge as prompt
            ],
            max_tokens=1024,
            temperature=0.4,
            repeat_penalty=1.5,
        )
        return resp["choices"][0]["message"]["content"]  # Extract model output
    except:
        print("Input text length:", len(before_merge_text) + len(system_message))
        print(before_merge_text)
    return "Model Error"


def prompt_response(before_merge_text):
    system_message = " You are a helping tearcher assistant. Your job is to explain codes, and not rewrite code."
    prompt = f"Tell me if the following code contains a bug or not. In case of bug, explain the bug.\n\n<code>\n{before_merge_text}\n<\code>"
    try:
        resp = llm(
            prompt=f"<|im_start|>system\n{system_message}.<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n.<|im_start|>assistant\n",
            max_tokens=1024,
            temperature=0.5,
            repeat_penalty=1.4,
            stop=["<|im_end|>"],
        )
        return resp["choices"][0]["message"]["content"]  # Extract model output
    except:
        print("Input text length:", len(before_merge_text) + len(system_message))
        print(json.dumps(resp, indent=2))
    return "Model Error"

In [103]:
start_time = time.time()
sample_data["model_output"] = sample_data["before_merge"].apply(prompt_response)
print("time to execute inference: ", time.time() - start_time)

Input text length: 487
{
  "id": "cmpl-109c7d2f-745d-49d6-9fa6-5ea6ecea4629",
  "object": "text_completion",
  "created": 1738533656,
  "model": "/Users/zanchitta/Developer/BugDetectiveAI/Bugdetectiveai/data/models/Ministral-3b-instruct.Q2_K.gguf",
  "choices": [
    {
      "text": "Here's how you can solve this:\n1. You know that the code is not correct and it doesn't work as expected. This should be done by a developer who knows what we want to do with each type of input. \n2. The function `process_input` takes an argument, which will either be the original data or another DataFrame. It then calls itself recursively on any other value (like empty array) and it doesn't pass their output into a dictionary or set. This can cause issues when trying to extract values from these lists:\n   - If you try `data_frame` instead, this will return the original data of all elements in your list as an iterator. \n   \n3. The function returns one value (like DataFrame), which is it's own structure 

#### Printing actual error message from sample data

In [86]:
# print(sample_data.loc[0,'traceback_type'])


# print("Full Traceback::::::")
# print(sample_data.loc[0,'full_traceback'])

sample_data.head()

,index,after_merge,before_merge,filename,full_file_code_after_merge,full_file_code_before_merge,function_name,url,source code and errors,full_traceback,traceback_type,before_merge_without_docstrings,after_merge_without_docstrings,before_merge_docstrings,after_merge_docstrings,path_to_snippet_before_merge,path_to_snippet_after_merge,input_length,model_output
0,189025,def _process_input(x):\n if isinsta...,def _process_input(x):\n if isinsta...,mars/dataframe/arithmetic/core.py,# Copyright 1999-2020 Alibaba Group Holding Lt...,# Copyright 1999-2020 Alibaba Group Holding Lt...,DataFrameBinOpMixin._process_input,https://github.com/mars-project/mars/issues/1286,"[{'piece_type': 'error message', 'piece_conten...","In [25]: df = md.DataFrame(mt.random.rand(10, ...",TypeError,def _process_input(x):\n if isinsta...,def _process_input(x):\n if isinsta...,[],[],buggy_snippets_files/848188c14a338af53fe76977d...,buggy_snippets_files/848188c14a338af53fe76977d...,397,No code has been provided.\n\nThe function `pr...


In [104]:
print(sample_data.loc[0, "before_merge"])

    def _process_input(x):
        if isinstance(x, (DATAFRAME_TYPE, SERIES_TYPE)) or np.isscalar(x):
            return x
        elif isinstance(x, pd.Series):
            return Series(x)
        elif isinstance(x, pd.DataFrame):
            return DataFrame(x)
        elif isinstance(x, (list, tuple, np.ndarray, TENSOR_TYPE)):
            return astensor(x)
        raise NotImplementedError


In [90]:
print(sample_data.loc[0, "after_merge"])

    def _process_input(x):
        if isinstance(x, (DATAFRAME_TYPE, SERIES_TYPE)) or pd.api.types.is_scalar(x):
            return x
        elif isinstance(x, pd.Series):
            return Series(x)
        elif isinstance(x, pd.DataFrame):
            return DataFrame(x)
        elif isinstance(x, (list, tuple, np.ndarray, TENSOR_TYPE)):
            return astensor(x)
        raise NotImplementedError


In [91]:
print(sample_data.loc[0, "model_output"])

No code has been provided.

The function `process_input` does not have a specific error handling mechanism. It doesn't handle any type of input like data, but it can perform the following tasks:

1. If isinstance(x): 
   - This is a DataFrame or DataGridView from Pandas. The value of this object is an array and it does not have a specific error handling mechanism.
2. Isteeped.
3. Isteeped.
4. None of the other types of input are allowed to be passed as input for processing. This could be because you're trying to perform operations like "transform" or "filter". 
5. If isinstance(x):
   - It doesn't have a specific error handling mechanism, so it can handle any type of data and is not required to perform the task.
6. Isteeped.
7. Isteeped.
8. None of the other types of input are allowed to be passed as input for processing. This could be because you're trying to perform operations like "transform" or "filter". 
9. Isteeped.
10. Isteeped.
11. Isteeped.
12. Isteeped.
13. Isteeped.
14. Iste

In [106]:
system_message = " You are a helping tearcher assistant. Your job is to explain codes, and not rewrite code."
prompt = f"Tell me if the following code contains a bug or not. In case of bug, explain the bug.\n\n<code>\n{easy_test}\n<\code>"

resp = llm(
    prompt=f"<|im_start|>system\n{system_message}.<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n.<|im_start|>assistant\n",
    max_tokens=1024,
    temperature=0.5,
    repeat_penalty=1.4,
    stop=["<|im_end|>"],
)
print(json.dumps(resp, indent=2))

{
  "id": "cmpl-222c2313-2903-4289-9767-a62952136bd7",
  "object": "text_completion",
  "created": 1738534298,
  "model": "/Users/zanchitta/Developer/BugDetectiveAI/Bugdetectiveai/data/models/Ministral-3b-instruct.Q2_K.gguf",
  "choices": [
    {
      "text": "The code you're looking at is not a bad one. It does the same as it seems to us:\n\n```python\ndef last_element_of_array(array):\n    return array[len(array)]\n```\nThis function takes an input data and returns its length, which gives them back into the list of values that can be accessed. It does this because there are only elements in the given range to find out how many times they're used. The `list()` method is a built-in Python dictionary where all keys have been entered as key-value pairs and it returns these numbers.\n\nThe function then calls itself recursively with each element from its list of values that are not yet returned by the 'finding'. This process continues until no more elements need to be found, which means 

In [107]:
resp["choices"][0]["message"]["content"]

KeyError: 'message'